In [1]:
import csv
import os

In [2]:
from nltk import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import utils
from tqdm import tqdm
import nltk
ps = nltk.stem.PorterStemmer()

In [32]:
facts_info = {}
facts = []
with open('../data/KnownLie/snopes/verified-claims.tsv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t')
    next(spamreader)
    for row in spamreader:
        facts.append(row[1])
        facts_info[row[1]] = remove_stopwords(utils.normalize_text(' '.join([row[0],row[1]])))

        #facts_info[row[1]] = remove_stopwords(normalize_text(row[1]))


In [4]:
from gensim.summarization.bm25 import get_bm25_weights
from gensim.summarization.bm25 import BM25
corpus = [f.split(" ") for f in facts]
results = BM25(corpus)

In [43]:
ground_truth = {}

all_claims = []

directory = '../data/KnownLie/snopes/fact-checking/'
i = 0
for root, dirs, files in os.walk(directory):
    for file in files:
        with open(directory + file) as csvfile:
            spamreader = csv.reader(csvfile, delimiter='\t')
            
            next(spamreader)
            for row in spamreader:
                text = row[1]
                scores = results.get_scores(text.split())
                idx = scores.index(max(scores))
                text = ' '.join(corpus[idx])
                
                title = row[0].lower().replace(',',' ')

                if title not in ground_truth:
                    ground_truth[title] = []
                all_claims.append(text)
                ground_truth[title].append(facts_info[text])
                i+=1


# GRAPH CREATION

In [6]:
import utils
import graphUtils

In [7]:
import networkx as nx

In [8]:
from itertools import chain
from nltk.corpus import wordnet
from sklearn.metrics.pairwise import cosine_similarity

G=nx.Graph()
K = 3

i = 0
nodes_labels = {}
claim_ids = {}
id_claim = {}


for claim in tqdm(ground_truth):
    node = remove_stopwords(utils.normalize_text(claim))
    i+=1

    node_name = str('CLM'+str(i))
    G.add_node(node_name , label= node_name, type='Claim')
    nodes_labels[node_name] = node_name
    claim_ids[node_name] = claim
    id_claim[claim] = node_name
                
    n_grams = [gr.replace(' ','_') for gr in graphUtils.find_all_n_grams(node,K)]
    n_grams = sorted(n_grams, key=lambda dist: len(dist),reverse = True)
    
    for tg in n_grams:
        #if len(tg.split('_')) == 1: tg = utils.ps.stem(tg)
        token = tg
        
            
        G.add_node(token,label=token, type='Token')
            
        if not G.has_edge(node_name,token): G.add_edge(node_name,token)

100%|██████████| 997/997 [00:01<00:00, 957.04it/s] 


In [11]:
i = 0
fact_ids = {}
id_fact = {}

for fact in tqdm(facts):
    node = remove_stopwords(utils.normalize_text(facts_info[fact]))
    i += 1
    name = str('FCT'+ str(i))
    
    fact_ids[name] = fact
    id_fact[fact] = name
    
    G.add_node(name,label = name, type='Fact')
    
    n_grams = [gr.replace(' ','_') for gr in graphUtils.find_all_n_grams(node,K)]
    n_grams = sorted(n_grams, key=lambda dist: len(dist),reverse = True)
    
    for tg in n_grams:
        if len(tg.split('_')) == 1: tg = utils.ps.stem(tg)
        token = tg
        
            
        if not G.has_node(token): continue

        if not G.has_edge(name,token):            G.add_edge(name,token)


  0%|          | 1/10381 [00:17<50:04:28, 17.37s/it]

scam scammer
fetus preborn
36 42 word
36 39 word
36 35 word
36 33 word


  0%|          | 2/10381 [00:44<58:33:55, 20.31s/it]

36 43 word
36 37 word


  0%|          | 7/10381 [01:41<35:00:36, 12.15s/it]

...

100%|█████████▉| 10366/10381 [42:17:45<03:42, 14.87s/it]

flight_disaster plane_crash


100%|█████████▉| 10373/10381 [42:19:10<01:31, 11.48s/it]

obama_presidency obama_administration


100%|██████████| 10381/10381 [42:20:39<00:00, 14.68s/it]


# BM25

In [ ]:
corpus = [facts_info[f].split() for f in facts]
results = BM25(corpus)

In [ ]:
import numpy as np
claim_facts_BM25 = {}
for claim in tqdm(ground_truth):
    cl_id = id_claim[claim]
    scores = results.get_scores(remove_stopwords(utils.normalize_text(claim)).split())
    arr = np.array(scores)
    topK = tops = arr.argsort()[::-1]
    claim_facts_BM25[cl_id] = [(id_fact[facts[idx]],scores[idx]) for idx in topK]

# RandomWalks

In [ ]:
import importlib
importlib.reload(graphUtils)

In [ ]:
import random
def random_walk(node,l):
    res = ''
    
    p = 0
    chosen = node
    
    res += str(chosen)

    while (p<l):
        chosen = random.sample([n for n in nx.neighbors(G,chosen)],1)[0]
        if G.nodes[chosen]['type'] in ['Claim','Fact','Token']:
            res += ' ' + str(chosen)
        p+=1
        
    return res


def generate_random_walks(k,l):
    rws = []
    
    for i in tqdm(range(0,k),position=0):
        for node in G.nodes():
            if len([n for n in nx.neighbors(G,node)]) == 0:
                continue
            if G.nodes[node]['type'] in ['Claim','Fact']:
                rws.append(random_walk(node,l))
    return rws


In [ ]:
docs = []
random_paths = generate_random_walks(200,l=30)
for p in random_paths:
    docs.append(p)

# WordEmbedding Models

In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from nltk.tokenize import word_tokenize

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from tqdm import tqdm 
tagged_data = []
for d in tqdm(docs,position=0):
    tagged_data.append(word_tokenize(d))


In [ ]:
%env PYTHONHASHSEED=0
max_epochs = 10
vec_size = 300

model = Word2Vec(size=vec_size, min_count=10, window=20, sg=0, seed=0, workers = 4)

model.build_vocab(tagged_data)

model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

print("Model is Ready")

In [ ]:
from gensim.models import FastText
%env PYTHONHASHSEED=0

max_epochs = 30
vec_size = 300

FTmodel = FastText(size=vec_size,
                min_count=10, 
                window=5,
                 sg=0,
                seed=0,
               workers = 4)


FTmodel.build_vocab(tagged_data)

FTmodel.train(tagged_data, total_examples=FTmodel.corpus_count, epochs=FTmodel.epochs)

print("Model Saved")

# Test

In [ ]:
gold_facts = {}
for claim in ground_truth:
    if claim not in gold_facts:
        gold_facts[claim] = []
    for fact in ground_truth[claim]:
        if fact in id_fact:
            gold_facts[claim].append(id_fact[fact])

In [ ]:
claim_facts = {}
for claim in tqdm(ground_truth):
    cl_id = id_claim[claim]
    filtered_facts = {}
    
    claim_facts[cl_id] = utils.distance_w2v (model,cl_id,fact_ids,20000)

In [ ]:
for KK in [1,3,5,10,20,50,10000]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MRR, hasP = 0,0,0

    for claim in claim_facts:
        if claim_ids[claim] not in gold_facts or len(gold_facts[claim_ids[claim]]) == 0: continue
        
        i+=1
        preds = [f for (f,j) in claims_pred[claim]][0:KK]
        golds = [f for f in gold_facts[claim_ids[claim]]]

        
        MAP += utils.MAP_K(golds,preds)
        MRR += utils.MRR(golds,preds)
        hasP += utils.HAS_POSITIVE(golds,preds)

        

    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MRR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)